In [1]:
import os
import cv2
import numpy as np
import os
from collections import Counter
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from PIL import Image 
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torch import optim , nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import numpy as np
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2



In [2]:
LEARNING_RATE=3e-4
BATCH_SIZE=5
EPOCHS=30
DATA_PATH="/kaggle/input/smoke-segmentation/ALL-2"
MODEL_SAVE_PATH = "/kaggle/working/unet_smoke_model.pth"
device="cuda" if torch.cuda.is_available() else "cpu"

In [8]:
def check_proportions():
    # Count images in each folder
    train_imgs = len([f for f in os.listdir(os.path.join(DATA_PATH, 'train')) if f.endswith('.jpg')])
    valid_imgs = len([f for f in os.listdir(os.path.join(DATA_PATH, 'valid')) if f.endswith('.jpg')])
    test_imgs  = len([f for f in os.listdir(os.path.join(DATA_PATH, 'test'))  if f.endswith('.jpg')])
    
    total = train_imgs + valid_imgs + test_imgs
    
    print(f"Train: {train_imgs} images ({train_imgs/total*100:.1f}%)")
    print(f"Valid: {valid_imgs} images ({valid_imgs/total*100:.1f}%)")
    print(f"Test:  {test_imgs} images ({test_imgs/total*100:.1f}%)")
    print(f"Total: {total} images")
    
    if 0.60 <= train_imgs/total <= 0.85:
        print("Proportion train OK (60-85%)")
    else:
        print("Proportion train inhabituelle")

# Call the function
check_proportions()

Train: 400 images (74.5%)
Valid: 40 images (7.4%)
Test:  97 images (18.1%)
Total: 537 images
Proportion train OK (60-85%)


In [10]:
def check_dimensions_match(folder='train', num_samples=50):
    folder_path = os.path.join(DATA_PATH, folder)
    images = [f for f in sorted(os.listdir(folder_path)) if f.endswith('.jpg')][:num_samples]
    
    print(f"\n=== DIMENSIONS {folder.upper()} ===")
    
    mismatches = []
    for img_file in images:
        img_path = os.path.join(folder_path, img_file)
        mask_file = img_file.replace('.jpg', '_mask.png')
        mask_path = os.path.join(folder_path, mask_file)
        
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"❌ Image corrompue: {img_file}")
            continue
        if mask is None:
            print(f"❌ Masque corrompu: {mask_file}")
            continue
        
        if img.shape[:2] != mask.shape:
            print(f"❌ {img_file}: Image {img.shape[:2]} ≠ Masque {mask.shape}")
            mismatches.append(img_file)
    
    if not mismatches:
        print(f"✅ Toutes les dimensions correspondent!")
        if images:
            img = cv2.imread(os.path.join(folder_path, images[0]))
            print(f"   Taille: {img.shape[:2]} (H, W)")
    
    return len(mismatches) == 0

# Run checks
check_dimensions_match('train')
check_dimensions_match('test')
check_dimensions_match('valid')


=== DIMENSIONS TRAIN ===
✅ Toutes les dimensions correspondent!
   Taille: (640, 640) (H, W)

=== DIMENSIONS TEST ===
✅ Toutes les dimensions correspondent!
   Taille: (640, 640) (H, W)

=== DIMENSIONS VALID ===
✅ Toutes les dimensions correspondent!
   Taille: (480, 852) (H, W)


True

In [13]:
def analyze_classes(folder='train'):
    folder_path = os.path.join(DATA_PATH, folder)
    masks = [f for f in sorted(os.listdir(folder_path)) if f.endswith('.png')]
    
    pixel_counts = Counter()
    all_classes = set()
    
    print(f"\n=== CLASSES {folder.upper()} ===")
    
    for mask_file in masks:
        mask = cv2.imread(f'/kaggle/input/smoke-segmentation/ALL-2/{folder}/{mask_file}', cv2.IMREAD_GRAYSCALE)
        unique, counts = np.unique(mask, return_counts=True)
        
        all_classes.update(unique)
        for cls, count in zip(unique, counts):
            pixel_counts[cls] += count
    
    print(f"Classes trouvées: {sorted(all_classes)}")
    
    total_pixels = sum(pixel_counts.values())
    print(f"\nDistribution:")
    for cls in sorted(pixel_counts.keys()):
        count = pixel_counts[cls]
        percentage = count / total_pixels * 100
        class_name = "Background" if cls == 0 else f"Smoke (classe {cls})"
        print(f"  {class_name}: {count:,} pixels ({percentage:.2f}%)")
    
    # Vérifier déséquilibre (sans compter le background)
    non_bg_counts = {k: v for k, v in pixel_counts.items() if k != 0}
    if len(non_bg_counts) > 0:
        smoke_pixels = sum(non_bg_counts.values())
        bg_pixels = pixel_counts[0]
        ratio = bg_pixels / smoke_pixels if smoke_pixels > 0 else 0
        
        print(f"\nRatio Background/Fumée: {ratio:.1f}:1")
        
        if ratio > 100:
            print("Fort déséquilibre - considérez les poids de classe")
        else:
            print("Déséquilibre acceptable")
    
    return pixel_counts

analyze_classes('train')


=== CLASSES TRAIN ===
Classes trouvées: [np.uint8(0), np.uint8(1), np.uint8(2), np.uint8(3)]

Distribution:
  Background: 315,254,880 pixels (90.52%)
  Smoke (classe 1): 864 pixels (0.00%)
  Smoke (classe 2): 5,438 pixels (0.00%)
  Smoke (classe 3): 33,020,553 pixels (9.48%)

Ratio Background/Fumée: 9.5:1
✅ Déséquilibre acceptable


Counter({np.uint8(0): np.int64(315254880),
         np.uint8(3): np.int64(33020553),
         np.uint8(2): np.int64(5438),
         np.uint8(1): np.int64(864)})

In [17]:
def check_empty_masks(folder='train'):
    folder_path = os.path.join(DATA_PATH, folder)
    masks = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    
    empty_masks = []
    
    print(f"\n=== MASQUES VIDES {folder.upper()} ===")
    
    for mask_file in masks:
        mask_path = os.path.join(folder_path, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if mask is None:
            print(f"Masque corrompu: {mask_file}")
            continue
        
        unique = np.unique(mask)
        
        # Si seulement background (classe 0)
        if len(unique) == 1 and unique[0] == 0:
            empty_masks.append(mask_file)
    
    print(f"Masques vides: {len(empty_masks)}/{len(masks)}")
    
    if empty_masks:
        print(f"{len(empty_masks)} masques sans fumée")
        print(f"   Exemples: {empty_masks[:5]}")
        
        # Vérifier si proportion élevée
        percentage = len(empty_masks) / len(masks) * 100
        if percentage > 10:
            print(f"{percentage:.1f}% de masques vides - est-ce normal?")
    else:
        print("✅ Tous les masques contiennent de la fumée")
    
    return empty_masks

# Exemple d'appel
check_empty_masks('train')
check_empty_masks('valid')
check_empty_masks('test')


=== MASQUES VIDES TRAIN ===
Masques vides: 0/400
✅ Tous les masques contiennent de la fumée

=== MASQUES VIDES VALID ===
Masques vides: 0/40
✅ Tous les masques contiennent de la fumée

=== MASQUES VIDES TEST ===
Masques vides: 0/97
✅ Tous les masques contiennent de la fumée


[]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dd

In [ ]:
class SmokeDataset(Dataset):
    def __init__(self, root_path, folder, transforms=None):
        self.root_path = root_path
        self.images = sorted([
            os.path.join(root_path, folder, i)
            for i in os.listdir(os.path.join(root_path, folder))
            if i.endswith(".jpg")
        ])
        self.masks = sorted([
            os.path.join(root_path, folder, i)
            for i in os.listdir(os.path.join(root_path, folder))
            if i.endswith(".png")
        ])
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.images[index]).convert('RGB')
        mask = Image.open(self.masks[index]).convert('L')
        
        image = image.resize((512, 512))
        mask = mask.resize((512, 512))
        
        img = np.array(image)
        mask = np.array(mask)
        
        mask[mask > 0] = 1
        
        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
    
        return img, mask
    

    def __len__(self):
        return len(self.images)


In [ ]:
train_transforms = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.5),
    A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
    ToTensorV2()
])

val_transforms = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
    ToTensorV2()
])

train_dataset = SmokeDataset(DATA_PATH, 'train', transforms=train_transforms)
val_dataset = SmokeDataset(DATA_PATH, 'valid', transforms=val_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels), 
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_op(x)

class DownSample(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.conv=DoubleConv(in_channels,out_channels)
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
    def forward(self,x):
        down=self.conv(x)
        p=self.pool(down)
        return down,p
class UpSample(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.up=nn.ConvTranspose2d(in_channels,in_channels//2,kernel_size=2,stride=2)
        self.conv=DoubleConv(in_channels,out_channels)
    def forward(self,x1,x2):
        x1=self.up(x1)
        x=torch.cat([x1,x2],1)
        return self.conv(x)
class UNet(nn.Module):
    def __init__(self,in_channels,num_classes):
        super().__init__()
        self.down_convolution_1=DownSample(in_channels,64)
        self.down_convolution_2=DownSample(64,128)
        self.down_convolution_3=DownSample(128,256)
        self.down_convolution_4=DownSample(256,512)
        
        self.bottle_neck=DoubleConv(512,1024)

        self.up_convolution_1=UpSample(1024,512)
        self.up_convolution_2=UpSample(512,256)
        self.up_convolution_3=UpSample(256,128)
        self.up_convolution_4=UpSample(128,64)


        self.out=nn.Conv2d(in_channels=64,out_channels=num_classes,kernel_size=1)
    def forward(self,x):
        down_1,p1=self.down_convolution_1(x)
        down_2,p2=self.down_convolution_2(p1)
        down_3,p3=self.down_convolution_3(p2)
        down_4,p4=self.down_convolution_4(p3)
        
        b=self.bottle_neck(p4)

        up_1=self.up_convolution_1(b,down_4)
        up_2=self.up_convolution_2(up_1,down_3)
        up_3=self.up_convolution_3(up_2,down_2)
        up_4=self.up_convolution_4(up_3,down_1)
        out=self.out(up_4)
        return out

In [ ]:
def dice_coef(y_pred, y_true, smooth=1e-6):
    y_pred = torch.sigmoid(y_pred)  # convert logits to probabilities
    y_true = y_true.float()
    intersection = (y_pred * y_true).sum(dim=(2,3))
    union = y_pred.sum(dim=(2,3)) + y_true.sum(dim=(2,3))
    dice = ((2 * intersection + smooth) / (union + smooth)).mean()
    return dice

In [ ]:
# model = UNet(in_channels=3, num_classes=1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(EPOCHS):
    model.train()
    train_running_loss = 0
    train_running_dice = 0

    for idx, (img, mask) in enumerate(train_dataloader):
        img = img.float().to(device)
        mask = mask.float().to(device)
        mask[mask > 0] = 1
        mask = mask.unsqueeze(1)  # (N,1,H,W)

        optimizer.zero_grad()
        y_pred = model(img)

        # BCE used for backprop
        loss = criterion(y_pred, mask)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.item()
        # Compute Dice as metric (no backward)
        train_running_dice += dice_coef(y_pred, mask).item()

    train_loss = train_running_loss / (idx + 1)
    train_dice = train_running_dice / (idx + 1)

    # Validation
    model.eval()
    val_running_loss = 0
    val_running_dice = 0
    with torch.no_grad():
        for idx, (img, mask) in enumerate(val_dataloader):
            img = img.float().to(device)
            mask = mask.float().to(device)
            mask = mask.unsqueeze(1)

            y_pred = model(img)
            loss = criterion(y_pred, mask)

            val_running_loss += loss.item()
            val_running_dice += dice_coef(y_pred, mask).item()

    val_loss = val_running_loss / (idx + 1)
    val_dice = val_running_dice / (idx + 1)

    print('-'*30)
    print(f'Train Loss: {train_loss:.4f} | Train Dice: {train_dice:.4f}')
    print(f'Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f}')

torch.save(model.state_dict(), MODEL_SAVE_PATH)